## Objective is to get Category of the product when the name of product is entered

In [7]:
# Importing Packages

import numpy as np 
import pandas as pd 
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
import gensim
#import fasttext
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB  
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.naive_bayes import MultinomialNB


In [4]:
#pip install python-Levenshtein

## Data Preprocessing

In [8]:
# Reading the dataset file
df = pd.read_csv('Categorical_Data2.csv')

In [9]:
#display csv file
df

,Unnamed: 0,Product name,Category
0,0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,Laptops
1,1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",Laptops
2,2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",Laptops
3,3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",Laptops
4,4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",Laptops
...,...,...,...
2830,3064,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,Air freshners
2831,3065,Lovin Air Freshener - 234 ml (Wild Romance),Air freshners
2832,3066,"ElectroCloud Car Air Freshener Vent Clip, Esse...",Air freshners
2833,3067,"Sugandhim Rose, Champa & Chameli Air Freshener...",Air freshners


In [10]:
#getting the required data from the dataset
df = df[['Product name', 'Category']]
df

,Product name,Category
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,Laptops
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",Laptops
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",Laptops
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",Laptops
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",Laptops
...,...,...
2830,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,Air freshners
2831,Lovin Air Freshener - 234 ml (Wild Romance),Air freshners
2832,"ElectroCloud Car Air Freshener Vent Clip, Esse...",Air freshners
2833,"Sugandhim Rose, Champa & Chameli Air Freshener...",Air freshners


In [11]:
df[df['Category'] == 'All in one PC']

,Product name,Category
1149,Lenovo IdeaCentre A340 23.8” FHD IPS All-in-On...,All in one PC
1150,"Lenovo IdeaCentre AIO 330 19.5"" All-in-One Des...",All in one PC
1151,HP All in One PC 20.7-inch(52.6 cm) FHD with A...,All in one PC
1152,Lenovo IdeaCentre AIO 330 19.5-inch All-in-One...,All in one PC
1153,"ASUS Vivo AiO V222, 21.5-inch (54.61 cms) FHD,...",All in one PC
...,...,...
1430,Radiant Electro 17 inch All in One Desktop Set...,All in one PC
1431,ASUS AIO / i3-10110U / 0 / HD Cam / 4GB / 1TB ...,All in one PC
1432,HP HP 570-p053in 19.45-inch All-in-One Desktop...,All in one PC
1433,Radiant Electro 17 inch All in One Desktop Set...,All in one PC


In [12]:
df[df['Category'] == 'Desktops']

,Product name,Category
856,Lenovo IdeaCentre A340 23.8-inch FHD All-in-On...,Desktops
857,Lenovo IdeaCentre 3 Desktop (AMD Athlon Silver...,Desktops
858,(Renewed) Dell Optiplex Business Class Perform...,Desktops
859,(Renewed) Dell Optiplex 3020 17-inch (43.18 cm...,Desktops
860,(Renewed) Dell Optiplex 380 17 inch (43.18 cms...,Desktops
...,...,...
1144,Zebronics Desktop Computer (Intel Core i7-860 ...,Desktops
1145,(Renewed) Dell Optiplex Business Class Perform...,Desktops
1146,TravisLappy Desktop PC CPU Computer CORE I3 32...,Desktops
1147,Capital Gadgets Desktop Computer Intel Core (i...,Desktops


In [13]:
df.Category.unique()

array(['Laptops', 'Phones', 'Printers', 'Desktops', 'All in one PC',
       'Adhesives', 'Mobile toilets', 'Art and craft', 'Smart TV',
       'Air freshners'], dtype=object)

In [14]:
# Checking for null values
df.isnull().sum()

Product name    0
Category        0
dtype: int64

In [15]:
# converting the categorical data to numerical data for further modeling
label_encoder = preprocessing.LabelEncoder()
df['Category'] = label_encoder.fit_transform(df['Category']) 
df

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2703998090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Category'] = label_encoder.fit_transform(df['Category'])


,Product name,Category
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,5
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",5
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",5
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",5
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",5
...,...,...
2830,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,1
2831,Lovin Air Freshener - 234 ml (Wild Romance),1
2832,"ElectroCloud Car Air Freshener Vent Clip, Esse...",1
2833,"Sugandhim Rose, Champa & Chameli Air Freshener...",1


In [16]:
# number of categories and their frequencies
df['Category'].value_counts()

6    301
7    298
4    293
2    286
0    286
9    285
8    280
5    278
3    268
1    260
Name: Category, dtype: int64

In [17]:
# dropping duplicate records, if any
df = df.drop_duplicates()
df = df.reset_index()

In [18]:
df.drop('index', axis =1, inplace = True)
df

,Product name,Category
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,5
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",5
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",5
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",5
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",5
...,...,...
2830,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,1
2831,Lovin Air Freshener - 234 ml (Wild Romance),1
2832,"ElectroCloud Car Air Freshener Vent Clip, Esse...",1
2833,"Sugandhim Rose, Champa & Chameli Air Freshener...",1


In [19]:
# function to get clean text from the product names

def clean_txt(sent):
    tokens = word_tokenize(re.sub('[^a-z0-9 ]',' ',sent.lower()))
    stop_updated = stopwords.words("english") + list(punctuation) + ["..."] + ["would", "could","told","subject"]
    text = [term for term in tokens if term not in stop_updated and
            len(term) > 2] 
    res = " ".join(text)
    return res

In [24]:
# obtaining cleaned titles from Product_Name
df['cleaned_title'] = df['Product name'].apply(clean_txt)
df

,Product name,Category,cleaned_title
0,HP 14 11th Gen Intel Core i5 Processor 14-inch...,5,11th gen intel core processor inch fhd laptop ...
1,"Dell Vostro 3400 14"" (35.56 cms) FHD Anti Glar...",5,dell vostro 3400 cms fhd anti glare display la...
2,"HP 15 (2021) Thin & Light Ryzen 3-3250 Laptop,...",5,2021 thin light ryzen 3250 laptop ram 1tb hdd ...
3,"HP 15 AMD Athlon 15.6"" (39.62cms) HD Laptop (S...",5,amd athlon 62cms laptop silver 3050u 4gb 1tb w...
4,"ASUS VivoBook 15 (2020), 39.6 cm HD, Dual Core...",5,asus vivobook 2020 dual core intel celeron n40...
...,...,...,...
2830,888 Combo of Rose-Lavender Camphor Cone 65 Gra...,1,888 combo rose lavender camphor cone grams air...
2831,Lovin Air Freshener - 234 ml (Wild Romance),1,lovin air freshener 234 wild romance
2832,"ElectroCloud Car Air Freshener Vent Clip, Esse...",1,electrocloud car air freshener vent clip essen...
2833,"Sugandhim Rose, Champa & Chameli Air Freshener...",1,sugandhim rose champa chameli air freshener sp...


## Word Embedding using Word2Vec

In [25]:
# loading Google's Pre Trainied Word2Vec Model

google_model = r'GoogleNews-vectors-negative300.bin.gz' 
embeddings = gensim.models.KeyedVectors.load_word2vec_format(google_model, binary=True)

In [26]:
# defining function to get real valued vector for each word in a record

def doc_vec(sent):    
    docs_vectors = pd.DataFrame()
    for doc in sent:
        temp = pd.DataFrame()
        words = doc.split(' ')
        for word in words:
            try:
                word2vec = embeddings[word]
                temp = temp.append(pd.Series(word2vec),ignore_index=True)
            except:
                pass
        doc_vector = temp.mean()
        docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
    return docs_vectors

In [27]:
# Using the UDF above to get vectors to perform modelling on
docs_vectors = doc_vec(df['cleaned_title'])
docs_vectors.head()

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = 

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_ve

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_ve

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_ve

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_ve

...

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_ve

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_ve

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = 

C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word2vec),ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
C:\Users\atiya\AppData\Local\Temp\ipykernel_19264\2169266837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = 

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.004306,-0.019270,0.025285,0.092551,-0.038125,0.078791,0.059637,-0.084561,0.099754,0.198153,...,-0.051100,0.007369,-0.069874,-0.013371,0.074296,-0.086159,-0.065119,-0.054188,-0.037442,-0.009839
1,0.040728,0.013768,-0.000529,0.065125,-0.027291,0.008894,0.044786,-0.085371,0.095154,0.121692,...,-0.060604,0.047398,-0.090929,0.007254,0.055426,-0.081706,-0.028233,-0.090044,-0.048402,-0.016026
2,0.008504,0.079508,-0.014398,-0.056925,0.026421,0.080355,0.032905,0.071859,0.148736,0.139301,...,-0.118869,0.073127,0.007521,0.022488,0.031928,-0.097005,-0.080153,-0.049235,-0.115614,-0.037111
3,0.002129,0.007195,-0.054058,0.028307,0.069187,-0.008762,-0.037218,-0.121691,0.155382,0.121392,...,-0.050483,-0.014363,-0.080736,-0.020759,0.023478,-0.119439,-0.022997,0.037018,-0.051058,-0.056736
4,-0.000513,0.009863,-0.019647,0.056429,-0.064480,0.103626,0.031820,-0.035307,0.097943,0.144833,...,-0.066004,0.083644,-0.009590,0.053206,-0.026292,-0.046936,-0.045346,-0.049927,-0.025102,-0.022311


In [28]:
docs_vectors.isnull().sum().sum()

300

In [29]:
nl = docs_vectors[docs_vectors[1].isnull()].index.tolist()
nl

[1956]

In [30]:
# dropping missing values from docs_vectors and corresponding category from df to maintain the same size
docs_vectors = docs_vectors.dropna()
df = df.drop(index = nl)

In [31]:
docs_vectors.shape

(2834, 300)

## Preparing Data for modelling for data embedded using Word2Vec

In [32]:
#defining X and Y for modelling
X = docs_vectors
y = df['Category'].values

In [33]:
#Splitting X and Y into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 2)

## Using different Classification Models to get the best model

In [34]:
#Using K Nearest Neighbors Classifier for classification
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)


#Predicting values for category based on the trained model using X test set
y_pred_knn = knn.predict(X_test)


C:\Users\atiya\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [35]:
# Confusion Matrix shows number of classes were classified correctly and incorrectly
confusion_matrix(y_test,y_pred_knn)

array([[ 82,   0,   0,   4,   0,   0,   3,   0,   0,   0],
       [  0,  81,   0,   0,   0,   0,   1,   0,   0,   0],
       [  0,   0,  54,   0,  27,   1,   0,   0,   0,   0],
       [  0,   0,   0,  80,   0,   0,   0,   0,   0,   0],
       [  0,   0,  22,   0,  51,   0,   0,   0,   0,   0],
       [  0,   0,   7,   0,   5,  71,   0,   0,   1,   0],
       [ 12,   1,   0,   6,   1,   1,  74,   0,   0,   0],
       [  0,   0,   1,   0,   0,   2,   0,  78,   1,   3],
       [  2,   0,   1,   1,   1,   1,   2,   0,  71,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 102]], dtype=int64)

In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred_knn))

              precision    recall  f1-score   support

           0       0.85      0.92      0.89        89
           1       0.99      0.99      0.99        82
           2       0.64      0.66      0.65        82
           3       0.88      1.00      0.94        80
           4       0.60      0.70      0.65        73
           5       0.93      0.85      0.89        84
           6       0.93      0.78      0.85        95
           7       1.00      0.92      0.96        85
           8       0.97      0.90      0.93        79
           9       0.97      1.00      0.99       102

    accuracy                           0.87       851
   macro avg       0.88      0.87      0.87       851
weighted avg       0.88      0.87      0.88       851



In [37]:
#Using Gaussian Naive Bayes Classifier for Classification
nb = GaussianNB()  
nb.fit(X_train, y_train)

#Predicting values for category based on the trained model using X test set
y_pred_nb = nb.predict(X_test)


In [38]:
#confusion matrix for Gaussian Naive Bayes Classifier
confusion_matrix(y_test, y_pred_nb)

array([[81,  0,  0,  2,  0,  0,  6,  0,  0,  0],
       [ 1, 81,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 45,  0, 27,  7,  0,  3,  0,  0],
       [ 1,  0,  0, 79,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 18,  0, 50,  4,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  5, 73,  0,  5,  1,  0],
       [10,  0,  0,  2,  0,  0, 79,  1,  3,  0],
       [ 1,  0,  0,  0,  0,  0,  0, 84,  0,  0],
       [ 1,  0,  0,  0,  0,  1,  0,  1, 76,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  1,  0, 99]], dtype=int64)

In [39]:
print(classification_report(y_test,y_pred_nb))

              precision    recall  f1-score   support

           0       0.84      0.91      0.87        89
           1       1.00      0.99      0.99        82
           2       0.71      0.55      0.62        82
           3       0.95      0.99      0.97        80
           4       0.61      0.68      0.65        73
           5       0.86      0.87      0.86        84
           6       0.93      0.83      0.88        95
           7       0.88      0.99      0.93        85
           8       0.94      0.96      0.95        79
           9       1.00      0.97      0.99       102

    accuracy                           0.88       851
   macro avg       0.87      0.87      0.87       851
weighted avg       0.88      0.88      0.88       851



In [40]:
#Using Multinomial Naive Bayes Classifier for Classification
#mnb = MultinomialNB()
#mnb.fit(X_train, y_train)

#Since X_train has negative values, the model cannot work

In [ ]:
#Using Random Forest Classifier for Classification
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train, y_train)

#Predicting values for category based on the trained model using X test set
y_pred_rf = rf.predict(X_test)


In [ ]:
#Confusion matrix for Random Forest Classifier
confusion_matrix(y_test, y_pred_rf)

In [ ]:
print(classification_report(y_test,y_pred_rf))

In [ ]:
#Using Decision Tree Classifier for Classification
clf_tree = DecisionTreeClassifier()
clf_tree.fit(X_train, y_train)

#Predicting values for category based on the trained model using X test set
y_pred_tree = clf_tree.predict(X_test)


In [ ]:
#Confusion matrix for Decision Tree Classifier
confusion_matrix(y_test, y_pred_tree)

In [ ]:
print(classification_report(y_test,y_pred_tree))

In [ ]:
#Using SVM model for classification
svc = svm.SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)


In [ ]:
confusion_matrix(y_test, y_pred_svc)

In [ ]:
print(classification_report(y_test,y_pred_svc))

In [ ]:
label_encoder.inverse_transform([0,1,2,3, 4,5,6,7,8,9])

In [ ]:
#Using Bagging Classifier for Classification
bag=BaggingClassifier()
bag.fit(X_train, y_train)
y_pred_bag = bag.predict(X_test)


#Confusion matrix
confusion_matrix(y_test, y_pred_bag)

In [ ]:
print(classification_report(y_test,y_pred_bag))

In [ ]:
#Using Bagging Classifier with Logisting Regression for Classification
lr = LogisticRegression()
bag_lr = BaggingClassifier(base_estimator=lr, random_state=7)
bag_lr.fit(X_train, y_train)
y_pred_bag_lr = bag_lr.predict(X_test)


#Confusion matrix
confusion_matrix(y_test, y_pred_bag_lr)

In [ ]:
print(classification_report(y_test,y_pred_bag_lr))

In [ ]:
#Using XGBoost Classifier for Classification
xgb=XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,
                  colsample_bytree=0.5,reg_alpha=0.000001 )
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)


#Confusion matrix
confusion_matrix(y_test, y_pred_xgb)

In [ ]:
print(classification_report(y_test,y_pred_xgb))

In [ ]:
#Using AdaBoost Classifier for Classification
weak_learner=DecisionTreeClassifier(max_depth=1,random_state=7)
adb = AdaBoostClassifier(base_estimator=weak_learner, n_estimators=10,random_state=7)
adb.fit(X_train, y_train)
y_pred_adb = adb.predict(X_test)


#Confusion matrix
confusion_matrix(y_test, y_pred_adb)

In [ ]:
print(classification_report(y_test,y_pred_adb))

In [ ]:
#Using LGBM Classifier for Classification
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred_lgb = lgbm.predict(X_test)


#Confusion matrix
confusion_matrix(y_test, y_pred_lgb)

In [ ]:
print(classification_report(y_test,y_pred_lgb))

In [ ]:
#Save test_set as excel file
#test_set.to_excel('ClassifierPerformance.xlsx', index = False)

### Using Grid Search CV to find the best values for hyperparameters

In [62]:
# SVM Hyperparameter Tuning

#setting up parameters with their possible input.
param_grid = {'C':[0.1,1,100,1000],'kernel':['rbf','poly','sigmoid','linear'],'degree':[1,2,3,4,5,6],'gamma':['scale','auto']}

#hyperparameter_tuning on basis of SVC model and scoring metric as 'accuracy'
grid = GridSearchCV(svm.SVC(),param_grid, scoring = 'accuracy')

#fiting the training data with best parameters
grid.fit(X_train,y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_grid_svm = grid.predict(X_test)

In [63]:
print(classification_report(y_test,y_pred_grid_svm))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        89
           1       0.99      0.99      0.99        82
           2       0.76      0.62      0.68        82
           3       0.95      1.00      0.98        80
           4       0.66      0.81      0.72        73
           5       0.96      0.96      0.96        84
           6       0.93      0.91      0.92        95
           7       1.00      1.00      1.00        85
           8       1.00      0.96      0.98        79
           9       1.00      1.00      1.00       102

    accuracy                           0.92       851
   macro avg       0.92      0.92      0.92       851
weighted avg       0.93      0.92      0.92       851



In [ ]:
# Random Forest Hyperparameter Tuning

#setting up parameters with their possible input.
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [5,6,7,8,9,10,100, 200, 300, 1000]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, n_jobs = -1, scoring = 'accuracy')

#fiting the training data with best parameters
grid_search.fit(X_train, y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_grid_rf = grid_search.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_grid_rf))

In [ ]:
# Decision Tree Hyperparameter Tuning

#setting up parameters with their possible input.
tree_para = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}


# Instantiate the grid search model
tree_grid = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = tree_para, n_jobs = -1, scoring = 'accuracy')

#fiting the training data with best parameters
tree_grid.fit(X_train, y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_tree_grid = tree_grid.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_tree_grid))

In [ ]:
# KNN Hyperparameter Tuning

#setting up parameters with their possible input.
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
# Create a based model

# Instantiate the grid search model
grid_knn = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = param_grid, n_jobs = -1, scoring = 'accuracy')

#fiting the training data with best parameters
grid_knn.fit(X_train,y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_knn_grid = grid_knn.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_knn_grid))

In [ ]:
# ADABoost Hyperparameter Tuning

#setting up parameters with their possible input.
param_grid={'n_estimators':[500, 1000, 2000],'learning_rate':[.001, 0.01, .1]}


# Instantiate the grid search model
adb_grid = GridSearchCV(estimator = AdaBoostClassifier(), param_grid = param_grid, scoring = 'accuracy',n_jobs = -1)

#fiting the training data with best parameters
adb_grid.fit(X_train, y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_adb_grid = adb_grid.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_adb_grid))

In [62]:
# Bagging Hyperparameter Tuning

#setting up parameters with their possible input.
param_grid = {'base_estimator__max_depth' : [1, 2, 3, 4, 5], 'max_samples' : [0.05, 0.1, 0.2, 0.5, 1, 3, 5, 7]}
                      
# Create a based model

# Instantiate the grid search model
bag_grid = GridSearchCV(estimator = BaggingClassifier(DecisionTreeClassifier()), param_grid = param_grid, scoring = 'accuracy', n_jobs = -1)

#fiting the training data with best parameters
bag_grid.fit(X_train, y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_bag_grid = bag_grid.predict(X_test)

In [63]:
print(classification_report(y_test,y_pred_bag_grid))

              precision    recall  f1-score   support

           0       0.69      0.66      0.68        89
           1       0.84      0.93      0.88        82
           2       0.67      0.52      0.59        82
           3       0.84      0.95      0.89        80
           4       0.53      0.70      0.60        73
           5       0.78      0.73      0.75        84
           6       0.73      0.69      0.71        95
           7       0.92      0.76      0.83        85
           8       0.85      0.80      0.82        79
           9       0.92      0.99      0.95       102

    accuracy                           0.78       851
   macro avg       0.78      0.77      0.77       851
weighted avg       0.78      0.78      0.78       851



In [64]:
# LGB Hyperparameter Tuning

#setting up parameters with their possible input.
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100, 300, 400],
    'lambda_l1': [1, 1.5, 2],
    'lambda_l2': [1, 1.5, 2]
    }

lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', learning_rate=0.01)
# Create a based model

# Instantiate the grid search model
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, n_jobs = -1)

#fiting the training data with best parameters
gsearch.fit(X_train, y_train)

#predicting category for the train set after tuning the hyperparameters of the model
y_pred_lgb_grid = gsearch.predict(X_test)

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1


In [65]:
print(classification_report(y_test,y_pred_lgb_grid))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85        89
           1       0.90      0.98      0.94        82
           2       0.69      0.60      0.64        82
           3       0.90      0.95      0.93        80
           4       0.60      0.70      0.65        73
           5       0.89      0.87      0.88        84
           6       0.89      0.79      0.84        95
           7       0.91      0.91      0.91        85
           8       0.92      0.91      0.92        79
           9       0.97      0.98      0.98       102

    accuracy                           0.86       851
   macro avg       0.85      0.85      0.85       851
weighted avg       0.86      0.86      0.86       851



In [67]:
# Listing names of the classifiers used and their scores
names = ['KNN Classifier', 'KNN Grid', 'Gaussian Naive Bayes Classifier', 'Random Forest Classifier', 'Random Forest Grid', 
         'Decision Tree Classifier', 'Decision Tree Grid',
         'SVC','SVC Grid Search CV','Ada Boost', 'Ada Boost Grid', 'Bagging', 'Bagging Grid', 'Light Gradient Boosting',
         'LGB Grid', 'Bagged with Logistic Regression', 'XGBoost']


accuracy_scores = [accuracy_score(y_test,y_pred_knn),
                   accuracy_score(y_test,y_pred_knn_grid),
                   accuracy_score(y_test,y_pred_nb), 
                   accuracy_score(y_test,y_pred_rf), 
                   accuracy_score(y_test,y_pred_grid_rf), 
                   accuracy_score(y_test,y_pred_tree), 
                   accuracy_score(y_test,y_pred_tree_grid), 
                   accuracy_score(y_test, y_pred_svc),
                   accuracy_score(y_test, y_pred_grid_svm),
                   accuracy_score(y_test, y_pred_adb),
                   accuracy_score(y_test, y_pred_adb_grid), 
                   accuracy_score(y_test, y_pred_bag), 
                   accuracy_score(y_test, y_pred_bag_grid),
                   accuracy_score(y_test, y_pred_lgb),
                   accuracy_score(y_test, y_pred_lgb_grid),
                   accuracy_score(y_test, y_pred_bag_lr), 
                   accuracy_score(y_test, y_pred_xgb)
                   #,accuracy_score(y_test, y_pred_xgb_grid)
                  ]


precision_scores = [precision_score(y_test, y_pred_knn, average = 'macro'),
                    precision_score(y_test, y_pred_knn_grid, average = 'macro'),
                    precision_score(y_test, y_pred_nb, average = 'macro'),
                    precision_score(y_test, y_pred_rf, average = 'macro'),
                    precision_score(y_test, y_pred_grid_rf, average = 'macro'),
                    precision_score(y_test, y_pred_tree, average = 'macro'),
                    precision_score(y_test, y_pred_tree_grid, average = 'macro'),
                    precision_score(y_test, y_pred_svc, average = 'macro'),
                    precision_score(y_test, y_pred_grid_svm, average = 'macro'),
                    precision_score(y_test, y_pred_adb, average = 'macro'),
                    precision_score(y_test, y_pred_adb_grid, average = 'macro'),
                    precision_score(y_test, y_pred_bag, average = 'macro'),
                    precision_score(y_test, y_pred_bag_grid, average = 'macro'),
                    precision_score(y_test, y_pred_lgb, average = 'macro'),
                    precision_score(y_test, y_pred_lgb_grid, average = 'macro'),
                    precision_score(y_test, y_pred_bag_lr, average = 'macro'),
                    precision_score(y_test, y_pred_xgb, average = 'macro')
                    #, precision_score(y_test, y_pred_xgb_grid, average = 'macro')
                   ]

recall_scores = [recall_score(y_test, y_pred_knn_grid, average = 'macro'),
                 recall_score(y_test, y_pred_knn, average = 'macro'),
                 recall_score(y_test, y_pred_nb, average = 'macro'),
                 recall_score(y_test, y_pred_rf, average = 'macro'),
                 recall_score(y_test, y_pred_grid_rf, average = 'macro'),
                 recall_score(y_test, y_pred_tree, average = 'macro'),
                 recall_score(y_test, y_pred_tree_grid, average = 'macro'),
                 recall_score(y_test, y_pred_svc, average = 'macro'),
                 recall_score(y_test, y_pred_grid_svm, average = 'macro'),
                 recall_score(y_test, y_pred_adb, average = 'macro'),
                 recall_score(y_test, y_pred_adb_grid, average = 'macro'),
                 recall_score(y_test, y_pred_bag, average = 'macro'),
                 recall_score(y_test, y_pred_bag_grid, average = 'macro'),
                 recall_score(y_test, y_pred_lgb, average = 'macro'),
                 recall_score(y_test, y_pred_lgb_grid, average = 'macro'),
                 recall_score(y_test, y_pred_bag_lr, average = 'macro'),
                 recall_score(y_test, y_pred_xgb, average = 'macro')
                 #, recall_score(y_test, y_pred_xgb_grid, average = 'macro')
                ]

f1_scores = [f1_score(y_test, y_pred_knn_grid, average = 'macro'),
             f1_score(y_test, y_pred_knn, average = 'macro'),
             f1_score(y_test, y_pred_nb, average = 'macro'),
             f1_score(y_test, y_pred_rf, average = 'macro'),
             f1_score(y_test, y_pred_grid_rf, average = 'macro'),
             f1_score(y_test, y_pred_tree, average = 'macro'),
             f1_score(y_test, y_pred_tree_grid, average = 'macro'),
             f1_score(y_test, y_pred_svc, average = 'macro'),
             f1_score(y_test, y_pred_grid_svm, average = 'macro'),
             f1_score(y_test, y_pred_adb, average = 'macro'),
             f1_score(y_test, y_pred_adb_grid, average = 'macro'),
             f1_score(y_test, y_pred_bag, average = 'macro'),
             f1_score(y_test, y_pred_bag_grid, average = 'macro'),
             f1_score(y_test, y_pred_lgb, average = 'macro'),
             f1_score(y_test, y_pred_lgb_grid, average = 'macro'),
             f1_score(y_test, y_pred_bag_lr, average = 'macro'),
             f1_score(y_test, y_pred_xgb, average = 'macro')
             #, f1_score(y_test, y_pred_xgb_grid, average = 'macro')
            ]

C:\Users\atiya.ahmed\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
# Getting a dataset from the lists above to easily compare the scores of the model used
scores_df = pd.DataFrame(zip(names, accuracy_scores, precision_scores, recall_scores, f1_scores), 
                           columns = ['Classifier', 'Accuracy Score', 'Precision Score', 'Recall Score', 'F1 Score'])
scores_df

,Classifier,Accuracy Score,Precision Score,Recall Score,F1 Score
0,KNN Classifier,0.874266,0.875963,0.873607,0.874898
1,KNN Grid,0.878966,0.878095,0.870689,0.871223
2,Gaussian Naive Bayes Classifier,0.877791,0.872162,0.874060,0.871010
3,Random Forest Classifier,0.727380,0.762777,0.725214,0.673498
4,Random Forest Grid,0.887192,0.878209,0.882059,0.878272
5,Decision Tree Classifier,0.759107,0.757874,0.752473,0.753680
6,Decision Tree Grid,0.759107,0.760337,0.755726,0.755628
7,SVC,0.922444,0.919982,0.919337,0.918044
8,SVC Grid Search CV,0.922444,0.919982,0.919337,0.918044
9,Ada Boost,0.568743,0.590602,0.569430,0.565351


In [72]:
#scores_df.to_excel('Scores after retraining Word2Vec model.xlsx')

#### After observing the performance of the models in fixed random_state and random text splittling, we concluded that only Gaussian Naive Bayes and SVM classifiers are stable in their preformance regardless of the randomness of the test data, hence we use both the classifiers henceforth.

## Getting predicted Category of entered Product Name

In [69]:
# UDF to clean the input text and get its vectorised form


def vect(sent):
    # sends text to clean using clean_txt()
    cln = clean_txt(sent)
    words = cln.split(' ')
    docs_vectors = pd.DataFrame()
    for doc in words:
        temp = pd.DataFrame()
        #print(word)
        try:
            word2vec = embeddings[doc]
            temp = temp.append(pd.Series(word2vec),ignore_index=True)
            
        except:
            pass
        doc_vector = temp.mean()
        docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)
    docs_vectors = docs_vectors.fillna(method='bfill')
    docs_vectors = docs_vectors.fillna(method='ffill')
    #print(docs_vectors)
    return docs_vectors

In [70]:
# UDF to get the class with most number of occurrences in the pred array
def most_likely(sent):
    class_ = []
    counter = 0
    new = list(label_encoder.inverse_transform(sent))
    for i in new:
        curr_frequency = new.count(i)
        if(curr_frequency > counter):
            counter = curr_frequency
            class_ = i
    return class_ 

In [73]:
#Get input from the user and sending that to UDF above
inp = vect(input('Enter title:'))

#predicting the category from user input
pred_svm = svc.predict(inp)
pred_nb = nb.predict(inp)

#printing the category predicted using UDF most_likely()
print('\nCategory SVM:', most_likely(pred_svm))
print('\nCategory Naive Bayes:', most_likely(pred_nb))

Enter title:All in one pc

Category SVM: Mobile toilets

Category Naive Bayes: Adhesives
